# MLonMCU

## Prerequisites

Before running this notebook, make sure to run the following steps:

1. Install the required software for your distribution:

   See main `README.md` file!
   
2. Clone this repository

   ```
   git clone TODO
   cd mlonmcu  # enter cloned repository
   ```
   
3. Setup and enter a new virtualenv (Python 3.7+) for using `mlonmcu`:

   ```
   Bla!
   source .venv/bin/activate
   ```
   
   Hint: You can also use a `conda` environment for this.
   
4. Install mlonmcu package

   ```
   python setup.py install
   ```
   
5. Install the required python packages:

   ``` 
   # pip install -r requirements.txt  # Done automatically via setup.py?
   pip install -r ipynb/requirements.txt
   ```
   
6. Launch this notebook in jupyter:

   ```
   jupyter notebook ipynb/Demo.ipynb
   ```

## Demonstration (Command Line)

### Preface

This guide should give an idea on the required steps to setup and use MLonMCU using the command line interface. Usage examples are provided as well.

### Initialize a MLonMCU environment

As MLonMCU requires a directory where dependencies, configurations, intermediate artifacts and results are stored, an environment needs to be initialized first. This also allows to use different configuration (e.g. enabled backends or supported features) for multiple environments.

Create and enter an environment diretcory first to initialize it afterwards:

In [6]:
# WARNING: execute this only once! Alternatively remove the environment directory beforehand to start from scratch.
!mkdir -p /tmp/mlonmcu_env
!mlonmcu init . --non-interactive # Do not add to envs?

Initializing ML on MCU environment
Selected target directory: /Users/philipp/src/work/prj/mlonmcu_open_source/mlonmcu/ipynb
The directory already exists!
The directory is not empty! - Aborting...
Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/bin/mlonmcu", line 33, in <module>
    sys.exit(load_entry_point('mlonmcu==0.1.0', 'console_scripts', 'mlonmcu')())
  File "/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/cli/main.py", line 55, in main
    args.func(args)
  File "/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/cli/init.py", line 83, in handle
    initialize_environment(
  File "/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/environment/init.py", line 109, in initialize_environment
    sys.exit(1)
NameError: name

When not using a notebook, the option `--non-interactive` can be omitted which allows to customize the initialization process interactively.

If the environment directory is not passed to the `mlonmcu` script, it will try to setup a `default` environment in the users home directory (On Ubuntu: `~/.config/mlonmcu/environments/default/`) instead. By providing a `--name` of the virtual environment if will be automatically registered in `~/.config/mlonmcy/environments.ini` for the current user which helps to find and distinguish multiple installed environments on a machine.

In the following the actual environment which should be used by the `mlonmcu` program has to be defined by the user either by defining the environment variable `MLONMCU_HOME` or by using the `-H` (`--home` or `--hint`) command line option. If none of this was done, the following rules are following to look for a suitable environment:

1. Check if `$MLONMCU_HOME` points to a valid environment
2. Use environment provided via the `-H` flag (path or registered name) if available
3. If the local working directory is the root of an environment, use this directory
4. If there is a default environment for the current user, fall back to this one instead

In [8]:
!export MLONMCU_HOME=/tmp/mlonmcu_env

Registered environment can be listed via the command `mlonmcu env list`. In addition there are possibilities to manually update entires in the environments file.

### Customizing the created MLonMCU environment (optional)

The environment was initialized by a template which can be found in the `templates/` directory of the repository chosen via the `--template` flag. Lets inspect the generated configuration file first:

In [9]:
!cat $MLONMCU_HOME/environment.yml

cat: /environment.yml: No such file or directory


The environment should work out of the box, but feel free to update the contents of the `environment.yml` file to achieve one of the following:

- Change the default paths used for dependencies, results, models,...
- Use different forks/versions of a repository
- Disable certain frameworks/backends to reduce the installation time in the next step.
- Define which features are supported for the given repositories
- Set default backends and targets
- Define a set of default configuration variables

To learn more about this, check out this document: `docs/ENVIRONMENTS.md` (TODO)

### Setting up all MLonMCU dependencies

Depending on the enabled backends and features, a different set of dependencies has to be installed. To reduce the management effort for the user, all these versions are installed and managed automatically using MLonMCU. To trigger this process, run the following in the command line. (This will take a very long time with all frameworks/targets/features enabled when invoked for the first time)

In [10]:
!mlonmcu setup

Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/bin/mlonmcu", line 33, in <module>
    sys.exit(load_entry_point('mlonmcu==0.1.0', 'console_scripts', 'mlonmcu')())
  File "/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/cli/main.py", line 55, in main
    args.func(args)
  File "/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/cli/setup.py", line 29, in handle
    with mlonmcu.context.MlonMcuContext(path=args.home, lock=True) as context:
  File "/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/context.py", line 229, in __init__
    self.environment = load_environment_from_file(
  File "/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/environment/loader.

Some things need to be considered here:
- Repositories are only cloned once, so upstream changes or updated urls/branches in the `environments.yml` are not detected automatically
- The flag `--rebuild` can be passed to `mlonmcu setup` to ensure that the setup routine of every dependency is triggered again. However this does not guarentee that all artifacts are properly updated
- To guarantee that the latest versions of a dependency are installed you can remove the `deps` directory manually or alternative use the provided `mlonmcu cleanup` utility. (Execute `mlonmcu cleanup --help` to learn more) 

### Adding models to MLonMCU

By default, the `mlonmcu-models` repository (TODO: url) is cloned to each environment as its contents are designed to perfectly integrate into the MLonMCU flow. In addition a location in the users home directory (`~/.config/mlonmcu/models/` for Ubuntu) is also added to the search path as defined in the `environments.yml`. Feel free to add further model manually or alternatively clone a model-zoo (e.g. TODO:arm) to a random location on the disc and updating the list of model-paths in the environments' config file.

To list all available models in a environment, the `mlonmcu models` subcommand can be used:

In [12]:
!mlonmcu models  # TODO: cleanup!

[2022-01-16 06:53:37,778]::/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/context.py:39::DEBUG - Starting lookup for mlonmcu environment
[2022-01-16 06:53:37,779]::/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/context.py:41::DEBUG - First checking in local working directory
[2022-01-16 06:53:37,779]::/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/context.py:47::DEBUG - Next checking environment variables
[2022-01-16 06:53:37,779]::/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/context.py:55::DEBUG - Looking for default environment for current user
[2022-01-16 06:53:37,779]::/usr/local/Caskroom/miniconda/base/envs/mlonmcu_open_source/lib/python3.8/site-packages/mlonmcu-0.1.0-py3.8.egg/mlonmcu/contex

The listed "Paths" should match the search paths defined in the `environments.yml` and are processed in the provided order. If there are multiple models of the same name, only the first one will be added to MLonMCU. By passing the `--detailed` flag to the command, additional information will be shown for each entry.

TODO: allow to search/filter for models by passing an extra arg

In addition to individual models, predefined model-groups are also available. I.e. the MLPerf Tiny benchmark defined here:

In [13]:
!cat $MLONMCU_HOME/models/groups.yml

cat: /models/groups.yml: No such file or directory


Feel free to manualy define further model-groups for your needs.

### Running the MLonMCU flow

The MLonMCU flow is a major part of the tool and therefore explained by several examples, starting with minimal use cases before discussing more complex examples later.

**Invoke a single model on the default backend/target**

In [ ]:
!mlonmcu flow run resnet

There exist multiple stages which can be processed. The subcommand passed to `mlonmcu flow` defines the stage where the flow should stop. If you are only interested in intermediate artifacts of the process, use one of the following:

```
mlonmcu flow load  # only process the model with the frontend
mlonmcu flow build  # invoke the chosen backend to generate code
mlonmcu flow compile  # (cross-)compile the code for the chosen target
```

Further commands are available to achive further behavior:

```
mlonmcu flow test  # use provided model-data to invoke the models comparing their outputs with a golden reference
```

**Explicitly specify the backend and target to use for the flow**

In [ ]:
!mlonmcu flow run resnet --backend tvmaot --target host_x86

**Enable certain features for all defined runs**

In [ ]:
!mlonmcu flow run resnet --backend tvmaot --target etiss_pulpino --feature unpacked_api

**Add additional configuration options to customize internal components**

In [ ]:
!mlonmcu flow run resnet --backend tvmaot --config tvmaot.target_device=arm_cpu

Instead of defining certain key-value pass on aeach command line, feel free to use the `vars:` property defined in the `environment.yml` to achieve the same result.

**Invoke multiple models on multiple backends/targets**

In [ ]:
!mlonmcu flow run resnet toycar --backend tvmaot --backend tflmi --target host_x86 --target etiss_pulpino

**Exploit parallelism to reduce the processing time**

In [ ]:
!mlonmcu flow run resnet toycar --backend tvmaot --backend tflmi --target host_x86 --target etiss_pulpino --parallel 4

???

**Inspecting and exporting results and intermediate artifacts**

TODO

In [ ]:
mlonmcu sess or mlonmcu export -l?

In [ ]:
!mlonmcu export --session 0 --run 0 run.zip
!mlonmcu export --session 0 session.zip
!mlonmcu export session/

### Further MLonMCU usage

In [ ]:
see below?

## Demonstration (Python API)

### Preface

While it is also possible to initialize and setup an MLonMCU environment via Python, it is highly recommended to use the provided command line interfaces for these steps. The following examples demonstrate how to use the MLonMCU flow from within a Python program which enables more customizations of the internal features via scripting.

### Open a MLonMCU context to load the environment

### Running the MLonMCU flow

### Low level APIs

Core components of MLonMC (Frontends, Backends, Targets) can also be invoked individually as demonstrated in the following examples: